In [1]:
import cv2


In [6]:
import cv2
import numpy as np
import pandas as pd
import os

# Step 1: Define paths for your images
image_paths = [
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2015.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2016.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2017.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2018.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2019.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2020.jpg"
]

# Step 2: Function to process and save each image
def process_image(image_path, output_csv):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Extract pixel data
    rows, cols = gray_image.shape
    data = []
    
    for i in range(rows):
        for j in range(cols):
            pixel_value = gray_image[i, j]
            data.append([i, j, pixel_value])  # Coordinates (i, j) and intensity
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['Row', 'Col', 'Intensity'])
    
    # Save to CSV
    df.to_csv(output_csv, index=False)
    print(f"Image {image_path} processed and saved to {output_csv}")

# Step 3: Process each image
output_dir = 'C:\\Users\\Hinal\\OneDrive\\Mtech Sem 1\\LAB WORK\\PDS\\OpenCV'
for idx, image_path in enumerate(image_paths):
    output_csv = os.path.join(output_dir, f'image_data_{idx + 1}.csv')
    process_image(image_path, output_csv)


Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2015.jpg processed and saved to C:\Users\Hinal\OneDrive\Mtech Sem 1\LAB WORK\PDS\OpenCV\image_data_1.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2016.jpg processed and saved to C:\Users\Hinal\OneDrive\Mtech Sem 1\LAB WORK\PDS\OpenCV\image_data_2.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2017.jpg processed and saved to C:\Users\Hinal\OneDrive\Mtech Sem 1\LAB WORK\PDS\OpenCV\image_data_3.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2018.jpg processed and saved to C:\Users\Hinal\OneDrive\Mtech Sem 1\LAB WORK\PDS\OpenCV\image_data_4.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2019.jpg processed and saved to C:\Users\Hinal\OneDrive\Mtech Sem 1\LAB WORK\PDS\OpenCV\image_data_5.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2020.jpg processed and saved to C:\Users\Hinal\OneDrive\Mtech Sem 1\LAB WORK\PDS\OpenCV\image_data_6.csv


In [8]:
# Step 3: Process each image and combine into a single DataFrame
combined_data = []

for idx, image_path in enumerate(image_paths):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    rows, cols = gray_image.shape
    for i in range(rows):
        for j in range(cols):
            pixel_value = gray_image[i, j]
            combined_data.append([idx + 1, i, j, pixel_value])  # Image index, coordinates, intensity

# Convert combined data to DataFrame
df_combined = pd.DataFrame(combined_data, columns=['Image', 'Row', 'Col', 'Intensity'])

# Save to a single CSV
df_combined.to_csv('C:\\Users\\Hinal\\OneDrive\\Mtech Sem 1\\LAB WORK\\PDS\\OpenCV\\combined_image_data.csv', index=False)
print("All images processed and combined into one CSV file.")


All images processed and combined into one CSV file.


In [2]:
pip install sqlalchemy


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.1 MB 1.3 MB/s eta 0:00:02
   -------------------- ------------------- 1.0/2.1 MB 1.4 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.1 MB 1.4 MB/s eta 0:00:01
   ----------------------------------- ---- 1.8/2.1 MB 1.6 MB/s eta 0:00:01
   ----------------------------------- ---- 1.8/2.1 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import pandas as pd
import os
import psycopg2
from sqlalchemy import create_engine

# Step 1: Define paths for your images
image_paths = [
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2015.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2016.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2017.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2018.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2019.jpg",
    "images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2020.jpg"
]


In [4]:
def process_image(image_path, output_csv, year):
    # Load the image
    image = cv2.imread(image_path)
    
    # Convert the image to grayscale (skip if already black and white)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Image dimensions and geo-coordinates mapping
    rows, cols = gray_image.shape
    lat_start, lat_end = 90, -90  # Latitude range from 90°N to 90°S
    lon_start, lon_end = -180, 180  # Longitude range from -180°W to 180°E

    # Calculate the latitude and longitude step per pixel
    lat_step = (lat_start - lat_end) / rows
    lon_step = (lon_end - lon_start) / cols

    # Prepare data for CSV
    data = []
    for i in range(rows):
        for j in range(cols):
            lat = lat_start - i * lat_step
            lon = lon_start + j * lon_step
            pixel_value = gray_image[i, j]
            data.append([year, lat, lon, pixel_value])

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(data, columns=['Year', 'Latitude', 'Longitude', 'Intensity'])
    df.to_csv(output_csv, index=False)
    print(f"Image {image_path} processed and saved to {output_csv}")


In [5]:
output_dir = 'output_csvs'  # Output directory for the CSV files
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

for idx, image_path in enumerate(image_paths):
    year = 2015 + idx  # Image year corresponds to 2015 through 2020
    output_csv = os.path.join(output_dir, f'CO2_budget_{year}.csv')
    process_image(image_path, output_csv, year)


Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2015.jpg processed and saved to output_csvs\CO2_budget_2015.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2016.jpg processed and saved to output_csvs\CO2_budget_2016.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2017.jpg processed and saved to output_csvs\CO2_budget_2017.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2018.jpg processed and saved to output_csvs\CO2_budget_2018.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2019.jpg processed and saved to output_csvs\CO2_budget_2019.csv
Image images/pilot_topdown_IS_NCE_CO2_Budget_grid_v1_2020.jpg processed and saved to output_csvs\CO2_budget_2020.csv


In [6]:
# Step 4: Combine all processed CSVs into one DataFrame and save
combined_data = []

for idx, image_path in enumerate(image_paths):
    year = 2015 + idx
    output_csv = os.path.join(output_dir, f'CO2_budget_{year}.csv')
    df = pd.read_csv(output_csv)
    combined_data.append(df)

# Concatenate all the DataFrames into one
df_combined = pd.concat(combined_data)

# Save combined data into a single CSV file
combined_csv_path = os.path.join(output_dir, 'combined_CO2_budget_data_2015_2020.csv')
df_combined.to_csv(combined_csv_path, index=False)
print(f"All images processed and combined into {combined_csv_path}")

All images processed and combined into output_csvs\combined_CO2_budget_data_2015_2020.csv


In [15]:
def create_database_if_not_exists(db_params):
    # Create connection string for PostgreSQL
    conn_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/postgres"
    
    # Create SQLAlchemy engine
    engine = create_engine(conn_string)
    
    # Create a connection to PostgreSQL
    with engine.connect() as conn:
        # Execute SQL to create database if it doesn't exist
        conn.execute(text(f"DO $$ BEGIN IF NOT EXISTS (SELECT 1 FROM pg_database WHERE datname = '{db_params['dbname']}') THEN CREATE DATABASE {db_params['dbname']}; END IF; END $$;"))
        print(f"Database {db_params['dbname']} is ready.")

def create_table_if_not_exists(engine, table_name):
    # Create table schema manually if needed
    with engine.connect() as conn:
        conn.execute(text(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            "Image" INTEGER,
            "Row" INTEGER,
            "Col" INTEGER,
            "Intensity" INTEGER
        )
        """))
        print(f"Table {table_name} is ready.")

def save_to_postgres(df, table_name, db_params):
    # Create connection string for PostgreSQL
    conn_string = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
    
    # Create SQLAlchemy engine
    engine = create_engine(conn_string)

    # Create database if it does not exist
    create_database_if_not_exists(db_params)
    
    # Create table schema if it does not exist
    create_table_if_not_exists(engine, table_name)
    
    # Log the DataFrame columns and first few rows
    print("DataFrame columns:", df.columns)
    print("First few rows:\n", df.head())
    
    # Save DataFrame to PostgreSQL
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Data saved to PostgreSQL table {table_name}")

# Parameters for the PostgreSQL connection
db_params = {
    'host': 'localhost',    # Your host
    'port': '5432',         # Default PostgreSQL port
    'user': 'postgres',     # Replace with your PostgreSQL username
    'password': '26022002', # Replace with your PostgreSQL password
    'dbname': 'NASA_U2'     # Replace with your PostgreSQL database name
}

# Step 6: Save combined data to PostgreSQL
save_to_postgres(df_combined, 'CO2_budget_data', db_params)

NameError: name 'text' is not defined